In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
file_proce_path = './Data/VersonData_2/traindf.csv'

In [3]:
traindf = pd.read_csv(file_proce_path,encoding='utf-8')
traindf.head()

,ID,Age,Gender,Education,Query List
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和 双沟 女生 中财网 首页 财经 周公 解梦 大全 查询 2345 曹云金 再讽 郭德纲...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,广州 厨宝 烤箱 世情 人情 雨送 黄昏 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想...
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石 之泪 耳机 盘锦 沈阳 旅顺 公交 辽宁 阜新 车牌 baidu k715 k716 ...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 壮观 衣服 网站 动漫 绘图 软件...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 怎么回事 麦当劳 旋风 勺子 吉林市 鹿...


## 使用n-gram测试

In [5]:
tfidf_vecone = TfidfVectorizer(ngram_range=(1,2),analyzer='word',max_features=3000)
tfidf_vecsec = TfidfVectorizer(ngram_range=(1,3),analyzer='word',max_features=1000)
tfidf_vecthd = TfidfVectorizer(ngram_range=(1,4),analyzer='word',max_features=1000)

## 分类AGE

In [6]:
agedf= traindf[['Age','Query List']]
agedf = agedf[agedf['Age']!=0]
X_age = agedf['Query List'].values.tolist()
Y_age = agedf['Age'].values.tolist()

In [7]:
cls_age = MultinomialNB()

In [8]:
def skfold_cv(x, y,vec_word, clf,shuffle=True, n_folds=5):
    stratifiedk_fold = StratifiedKFold(n_splits=n_folds, shuffle=shuffle)
    history_score=[]
    vec_word.fit(x)
    for train_index, test_index in stratifiedk_fold.split(x, y):
        X_train, X_test = x[train_index], x[test_index]
        y_train,y_test = y[train_index],y[test_index]
        clf.fit(vec_word.transform(X_train),y_train)
        history_score.append(clf.score(vec_word.transform(X_test), y_test)) 
    return history_score 

In [9]:
age_score_one = skfold_cv(np.array(X_age),np.array(Y_age),tfidf_vecone,cls_age)
print(age_score_one)

[0.5397834155269714, 0.5368619076672768, 0.5430416433619769, 0.5443128082574872, 0.5426435437115394]


In [10]:
age_score_two = skfold_cv(np.array(X_age),np.array(Y_age),tfidf_vecsec,cls_age)
print(age_score_two)

[0.529208399003508, 0.5358958714663412, 0.5238724767376824, 0.5248385620582702, 0.5313024462187865]


内存不够，我的天 可能4000太多了

## 分类性别

In [11]:
genderdf = traindf[['Age','Query List','Gender']]
genderdf = genderdf[genderdf['Gender']!= 0]
X_gender = genderdf['Query List'].values.tolist()
Y_gender = genderdf['Gender'].values.tolist() 
Y_ageflg = genderdf['Age'].values.tolist() 

In [12]:
def skfold_gender_cv(x, y,y_flag,vec_word, clf,shuffle=True, n_folds=5):
    stratifiedk_fold = StratifiedKFold(n_splits=n_folds, shuffle=shuffle)
    history_score=[]
    vec_word.fit(x)
    for train_index, test_index in stratifiedk_fold.split(x, y_flag):
        X_train, X_test = x[train_index], x[test_index]
        y_train,y_test = y[train_index],y[test_index]
        clf.fit(vec_word.transform(X_train),y_train)
        history_score.append(clf.score(vec_word.transform(X_test), y_test)) 
    return history_score 

In [13]:
cls_gender = MultinomialNB()

In [14]:
del tfidf_vecsec

In [15]:
gender_history_score = skfold_gender_cv(np.array(X_gender),np.array(Y_gender),np.array(Y_ageflg),tfidf_vecone,cls_gender)
print(gender_history_score)

[0.800071530758226, 0.8001124112206837, 0.7928867085696766, 0.7962385649307507, 0.7965859143412042]


## 分类学历

In [16]:
educationdf = traindf[['Education','Query List']]
educationdf = educationdf[educationdf['Education']!=0]
X_education = educationdf['Query List'].values.tolist()
Y_education = educationdf['Education'].values.tolist()

In [17]:
cls_education = MultinomialNB()

In [18]:
education_history_score = skfold_cv(np.array(X_education),np.array(Y_education),tfidf_vecone,cls_education)
print(education_history_score)

[0.5674528821778905, 0.5710900473933649, 0.5664370349958666, 0.5707198765295999, 0.5660658177608732]


## 平均准确率

In [19]:
print('age score  ',np.mean(age_score_one ))
print('gender score  ',np.mean(gender_history_score))
print('education score  ',np.mean(education_history_score))

age score   0.5413286637050503
gender score   0.7971790259641083
education score   0.568353131771519


In [21]:
average_score = (np.mean(age_score_one) + np.mean(gender_history_score)+ np.mean(education_history_score) )/3
print('average_score ', average_score)

average_score  0.6356202738135592
